# Task 4 - Using VQE to find lowest eigenvalue

### Karthik Krishnakumar, *Msc. Physics, BITS Pilani*

## **Question**:

Find the lowest eigenvalue of the following matrix:

$$
\begin{bmatrix}
1 & 0 & 0 & 0  \\
0 & 0 & -1 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}.
$$

using VQE-like circuits, created by yourself from scratch.

## **Resources**:

I found the following resources very important to develop my understanding of the Algorithm 
- [David Khachatryan's blog on VQE](https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/variational_quantum_eigensolver.ipynb)
- [Michel Stechly's blog on VQE](https://www.mustythoughts.com/variational-quantum-eigensolver-explained)



## **Basic Idea**

Before we start actually targetting the problem at hand let's get a broad understanding of the VQE Algorithm, the Algorithm works on the idea that the lowest expectation value of a given Hamiltonian occurs at the lowest energy eigenvector. This implies that if we search the entire state space for the lowest expectation value we will find the lowest eigenvalue in the progress. In general,this would be a very tedious task that will require some form of gradient descent scheme to simplify and optimize the process of Parameter finding.But for this 4 dimensional matrix it is computationally feasible to search the complete state space to find the minima.To find the expectation value of the Hamiltonian we decompose the Hamiltonian into Pauli Bases since their expectation values can be calculated, we also design an ansatz that covers the entire state space


I have broken down the algorithm into the following broad sections

1.   Decomposing the Hamiltonian into Pauli Bases.
2.   Building circuit for Ansatz design
3.   Finding expectation value in Paul Bases
4.   Identifying minimum energy state
5.   Testing results obtained.


#  1) Decomposing Hamiltonian into Pauli bases


We know that for one qubit the Pauli Matrices $I,X,Y,Z$ forms an orthogonal basis. Similarly in our problem the 4-dimensional matirx can be decomposed into the Pauli Bases as well which is made up of $II,XX,YY,ZZ,XY......$ forms a basis. 

$$
\sigma_0 = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} \quad
\sigma_x = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \quad
\sigma_y = \begin{pmatrix} 0 &-i \\ i & 0 \end{pmatrix} \quad
\sigma_z = \begin{pmatrix} 1 & 0 \\ 0 &-1 \end{pmatrix}
$$

$$ H = \sum_{i_{1},\ldots,i_{n} = \{0,x,y,z\}}h_{i_{1},\ldots,i_{n}}\cdot \frac{1}{2^{n}}\sigma_{i_1}\otimes\ldots\otimes\sigma_{i_n} $$



We are required to find the coefficients $h_{i_{1},\ldots,i_{n}}$ 

For the problem at hand we have $n=2$, 

$$
H = \sum_{ij} \frac{1}{4} h_{ij} \hspace{0.5em} \sigma_i \otimes \sigma_j
$$

We find the coefficients $h_{ij}$ using the Gram-Schmidt product and the components $h_{ij}$ are:

$$
h_{ij} = \frac{1}{4} \mathrm{Tr} \bigg[ (\sigma_i \otimes \sigma_j) \cdot H \bigg]
$$

Therefore we create a function `decompose_pauli` performs this decomposition for us.










In [63]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import *
from itertools import product
from scipy.optimize import minimize



In [64]:
H = np.array([[1,0,0,0],[0,0,-1,0],[0,-1,0,0],[0,0,0,1]])

sI = np.eye(2, 2, dtype=complex)
sX = np.array([[0, 1], [1, 0]], dtype=complex)
sZ = np.array([[1, 0], [0,-1]], dtype=complex)
sY = np.array([[0,1j],[-1j,0]])


pauli = {'I': sI,
     'X': sX,
     'Y': sY,
     'Z': sZ}

In [226]:
def decompose_pauli(H, pauli):
    """ Decompose a general Hamiltonian into the pauli matrices,in this case II,XX,YY,ZZ..
    
    Args:
        H : Matrix we want to decompose.
        pauli (dict): basis matrices with names as keys
   
   Output:
       coefficients (dict):  values of components along various Pauli basis matrices"""
       
       
    assert np.allclose(H, H.conj().T), "The Hamiltonian is not Hermitian"
    assert len(H) == len(H[0]), "Matrix is not a square matrix"
     
    repeat = int(np.log2(len(H)))
    coefficients = {}
    
    for (name_1, pauli_1),(name_2, pauli_2) in product(basis.items(), repeat=repeat):           
        coefficients[name_1 + name_2] = np.trace(1/4.*np.kron(pauli_1,pauli_2) @ H)
    
    return { key : val for key,val in coefficients.items() if val != 0}

**Testing the Pauli Decompose function**

Let's test the function created above by decomposing the Identity matrix and reviewing the solution obtained

In [227]:
Hamil= np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
components = decompose_pauli(Hamil, pauli)

for name, comp in components.items():
    print(f"  {name} : {comp}")


  II : (1+0j)


Therefore,we can see that the Pauli decompose function works,now let's find the coefficients for the actual given Hamiltonian 

In [228]:
components = decompose_pauli(H, pauli)

for name, comp in components.items():
    print(f"  {name} : {comp}")


  II : (0.5+0j)
  XX : (-0.5+0j)
  YY : (-0.5+0j)
  ZZ : (0.5+0j)


Therefore we can write the following equation decomposing the given Hamiltonian in the following form.
$$H=0.5[I\otimes I]-0.5[X\otimes X]-0.5[Y\otimes Y]+0.5[Z\otimes Z]$$

#  2) Building circuit for Ansatz Initialization

Now we must design the ansatz for the circuit we choose the following $$[R_{X}(\theta)\otimes I][CNOT_{12}][H\otimes I]$$



In [229]:
def ansatz_preparation(circuit,parameters):
    
    q=circuit.qregs[0]
    circuit.h(q[0])
    circuit.cx(q[0],q[1])    
    circuit.rx(parameters,q[0])
    
    return circuit

Testing the ansatz preparation circuit 

In [230]:
q = QuantumRegister(2)
circuit = QuantumCircuit(q)

my_circuit = ansatz_preparation(circuit,2)
my_circuit.draw()

┌───┐     ┌───────┐
q4560_0: ┤ H ├──■──┤ RX(2) ├
         └───┘┌─┴─┐└───────┘
q4560_1: ─────┤ X ├─────────
              └───┘

Now that we have designed the ansatz preparation circuit, we can move on to the next part of the circuit which is finding the expectation value in the required Pauli Bases.

# 3) Finding the expectation value in the Pauli Bases.

By default Qiskit performs the measurement in the Computational Z basis for the other basis we require to do a transformation into the respective basis, we know that the transformation into X basis requires a Hadamard gate simillarly we the transformation for Y is given by S dagger
.$$Y_{gate}= \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & -i \\ 1 & i \end{pmatrix}\quad H_{gate}= \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}$$


Since we only have the pauli bases $II,XX,YY,ZZ$ in our decomposition we only need to crerate the tranformation matrices for those measurement basis. The bases are defined below.

$$ ZZ : [I_{1}]\otimes[I_{2}]$$
$$ XX : [Hadamard_{1}]\otimes [Hadamard_{2}]$$
$$ YY : [Y_{gate 1}]\otimes [Y_{gate 2}]$$




In [231]:
def vqe_circuit(parameters, measure):
    """
    Creates a device ansatz circuit for optimization.
    :param parameters_array: list of parameters for constructing ansatz state that should be optimized.
    :param measure: measurement type. E.g. 'Z' stands for Z measurement.
    :return: quantum circuit."""
    
    # Ansatz preparation
    
    
    q = QuantumRegister(2)
    c= ClassicalRegister(2)
    circuit = QuantumCircuit(q,c)
    
    circuit=ansatz_preparation(circuit,parameters)
    circuit.barrier()
    
    
    # measurement
    if measure == 'ZZ':
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'XX':
        circuit.u2(0, np.pi, q[0])
        circuit.u2(0, np.pi, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    elif measure == 'YY':
        circuit.u2(0, np.pi/2, q[0])
        circuit.u2(0, np.pi/2, q[1])
        circuit.measure(q[0], c[0])
        circuit.measure(q[1], c[1])
    else:
        raise ValueError('Not valid input for measurement: input should be "XX" or "YY" or "ZZ"')

    return circuit

Testing the Measurement circuit for all basis measurement

In [232]:
firt=vqe_circuit(2,'ZZ')
firt.draw()

┌───┐     ┌───────┐ ░ ┌─┐   
q4561_0: ┤ H ├──■──┤ RX(2) ├─░─┤M├───
         └───┘┌─┴─┐└───────┘ ░ └╥┘┌─┐
q4561_1: ─────┤ X ├──────────░──╫─┤M├
              └───┘          ░  ║ └╥┘
c4538_0: ═══════════════════════╩══╬═
                                   ║ 
c4538_1: ══════════════════════════╩═

In [233]:
firty=vqe_circuit(2,'YY')
firty.draw()

┌───┐     ┌───────┐ ░ ┌────────────┐┌─┐   
q4562_0: ┤ H ├──■──┤ RX(2) ├─░─┤ U2(0,pi/2) ├┤M├───
         └───┘┌─┴─┐└───────┘ ░ ├────────────┤└╥┘┌─┐
q4562_1: ─────┤ X ├──────────░─┤ U2(0,pi/2) ├─╫─┤M├
              └───┘          ░ └────────────┘ ║ └╥┘
c4539_0: ═════════════════════════════════════╩══╬═
                                                 ║ 
c4539_1: ════════════════════════════════════════╩═

In [234]:
firt=vqe_circuit(2,'XX')
firt.draw()

┌───┐     ┌───────┐ ░ ┌──────────┐┌─┐   
q4563_0: ┤ H ├──■──┤ RX(2) ├─░─┤ U2(0,pi) ├┤M├───
         └───┘┌─┴─┐└───────┘ ░ ├──────────┤└╥┘┌─┐
q4563_1: ─────┤ X ├──────────░─┤ U2(0,pi) ├─╫─┤M├
              └───┘          ░ └──────────┘ ║ └╥┘
c4540_0: ═══════════════════════════════════╩══╬═
                                               ║ 
c4540_1: ══════════════════════════════════════╩═

**Finding the expectation value for a given Pauli basis**

To obtain the expectation value of the energy we first need to perform the transformation to the necessary basis and then perform measurement in the computational basis we will obtain the following measurements as output $00,01,10,11$ the outcomes $00,11$ correspond to a classical measurement readout of +1 whereas the measurements $10,01$ correspond to a classical readout of -1

In [235]:
def expectation_Pauli(parameters, measure):
    # measure
    if measure == 'II':
        return 1
    elif measure == 'ZZ':
        circuit = vqe_circuit(parameters, 'ZZ')
    elif measure == 'XX':
        circuit = vqe_circuit(parameters, 'XX')
    elif measure == 'YY':
        circuit = vqe_circuit(parameters, 'YY')
    else:
        raise ValueError('Not valid input for measurement: input should be "II" or "XX" or "ZZ" or "YY"')
    
    shots = 8192
    backend = BasicAer.get_backend('qasm_simulator')
    job = execute(circuit, backend, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    # expectation value estimation from counts
    expectation_value = 0
    for measure_result in counts:
        
        if measure_result == '11':
            sign = +1
        elif measure_result == '01':
            sign = -1
        elif measure_result == '10':
            sign = -1
        elif measure_result == '00':
            sign = +1
        expectation_value += sign * counts[measure_result] / shots
        
    return expectation_value

**Testing the expectation value for different Pauli measurements**

By setting the value of params equal to 0 we get the bell state $\frac{1}{\sqrt{2}}[|00\rangle + |11\rangle]$ measuring in the ZZ basis we must obtain either 00 or 11 as the outcome both of which result in a positive sign for ZZ measurement.Therefore we must have an expectation value of 1


In [236]:
param=0
expect= expectation_Pauli(param,'ZZ')
expect

1.0

Now we have defined a function which finds the expectation value of measurement in each Pauli Basis necessary,given below is a function which combines all the expectation values to find the energy value of the Hamiltonian for a given parameter

In [237]:

def vqe(parameters):
        
    # quantum_modules
    quantum_module_I = components['II'] * expectation_Pauli(parameters, 'II')
    quantum_module_Z = components['ZZ'] * expectation_Pauli(parameters, 'ZZ')
    quantum_module_X = components['XX'] * expectation_Pauli(parameters, 'XX')
    quantum_module_Y = components['YY'] * expectation_Pauli(parameters, 'YY')
    
    # summing the measurement results
    classical_adder = quantum_module_I + quantum_module_Z + quantum_module_X + quantum_module_Y
    classical_adder= classical_adder.real
    return classical_adder

# 4) Obtaining minimum energy state

Now that we have defined the function we now need to minimize the energy values for all possible values of the parameter,since the parameter represents rotation all values between $0$ and $2\pi$ must be considered, there are two ways of finding the minima

1. Using a gradient descent scheme to identify the minimum value of energy
2. Plotting the function for all values of the parameter and identifying the minima

We will see both these methods below 


**Using gradient descent scheme(Powell)**

In [238]:
parameters_array = np.pi
tol = 1e-3 # tolerance for optimization precision.

vqe_result = minimize(vqe, parameters_array, method="Powell", tol=tol)
print('The estimated ground state energy from VQE algorithm is: {}'.format(vqe_result.fun))
print('The value of parameter for which minima is obtained is : {}'.format(vqe_result.x))


The estimated ground state energy from VQE algorithm is: -1.0
The value of parameter for which minima is obtained is : 3.137831298570027


**Plotting the function and obtaining minima**

In [ ]:
theta_range = np.linspace(0, 2 * np.pi, 128)
average_range=[]
for x in theta_range:
    val = vqe(x)
    average_range.append(val)
plt.plot(theta_range,average_range)
plt.title(r"Variation of Energy wrt to parameter $", fontsize=16)
plt.xlabel(r"$\theta$", fontsize=14)
plt.ylabel(r"Energy", fontsize=14)
plt.show()

In [240]:
print(f"Smallest eigenvalue from VQE: {np.round(np.min(average_range),4)} with theta = {np.round(theta_range[np.argmin(average_range)], 2)}")

Smallest eigenvalue from VQE: -0.9996 with theta = 3.12



# 5) Testing result obtained

Now that we have obtained the result we need to verify the credibility of the outcome we can compare the answer received to a result obtained using a classical eigenvalue finding scheme


In [241]:
print(f"Smallest eigenvalue calculated classically = {np.round(np.min(np.linalg.eigh(H)[0]),4)}")

Smallest eigenvalue calculated classically = -1.0


Therefore, we can see that the result obtained using the gradient descent scheme exactly matches the result obtained from a classical eigenvalue finding tool.

The graphical method employed above can get extremely intensive with a greater no. of parameters, the above algorithm can be extended to more Hamiltonians with all pauli combinations considered.

**Scope for further work**

1. Extending the Algorithm to a general $N=2^{n}$ dimensional hermitian hamiltonian matrix
2. Decomposing more generally into all possible Pauli combinations
3. Using different classical optimizers to identify their effect on the final result
4. Using noisy models of qubits to identify viability of the Algorithm on NISQ era computers